# **Bayesian GraphSAGE**

## ***Environment settings***

In [1]:
!rm -r /content/sample_data

In [2]:
# Remove existing JAX installations
!pip uninstall -y -qq jax jaxlib jax-cuda12-plugin scipy pyarrow gymnasium numpy

In [3]:
# Install JAX with CUDA support
!pip install -qq --upgrade "jax[cuda12]"
!pip install scipy gymnasium==0.29.0
!pip install tensorboard
!pip install tensorboard-plugin-profile

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 57.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.7/153.7 MB 12.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.1/80.1 MB 23.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 105.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 58.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 81.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 MB 48.5 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 4.4.1 requires pyarrow>=21.0.0, which is not installed.
pettingzoo 1.24.0 requires gymnasium>=0.28.0, which is not i

In [4]:
# Install core dependencies
%pip -qq install --upgrade jax jaxlib flax optax orbax-checkpoint grain
%pip -qq install numpy matplotlib scipy
%pip -qq install torch torchvision
%pip -qq install datasets 
%pip -qq install msgpack requests tqdm
%pip -qq install bitsandbytes
%pip -qq install jraph
%pip -qq install networkx
%pip -qq install ogb
%pip -qq install pyarrow
%pip -qq install db-sqlite3
%pip -qq install pandas polars
%pip -qq install bitsandbytes numpyro langdetect
%pip -qq install xprof
%pip -qq install jax2onnx
%pip -qq install onnx onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.0/488.0 kB 8.9 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 602.4/602.4 kB 19.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dopamine-rl 4.1.2 requires gymnasium>=1.0.0, but you have gymnasium 0.29.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 42.3 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.26.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installe

In [5]:
# Install Git LFS for large files
!apt install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.3).
0 upgraded, 0 newly installed, 0 to remove and 98 not upgraded.


In [6]:
import IPython
print("Rebooting kernel... Please wait 5-10 seconds.")
IPython.Application.instance().kernel.do_shutdown(restart=True)
!pip install --upgrade pip

Rebooting kernel... Please wait 5-10 seconds.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 22.0 MB/s eta 0:00:0000:010:01
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [1]:
!pip install -qq --upgrade "jax[cuda12]"

In [2]:
print(100*"-")
%pip show jax
print(100*"-")
%pip show jaxlib
print(100*"-")
%pip show jax-cuda12-plugin
print(100*"-")
%pip show flax
print(100*"-")
%pip show optax
print(100*"-")
%pip show torch
print(100*"-")
%pip show torchvision
print(100*"-")
%pip show orbax-checkpoint
print(100*"-")
%pip show numpy
print(100*"-")
%pip show tqdm
print(100*"-")
%pip show datasets
print(100*"-")
%pip show msgpack
print(100*"-")
%pip show bitsandbytes
print(100*"-")
%pip show jraph
print(100*"-")
%pip show networkx
print(100*"-")
%pip show ogb
print(100*"-")
%pip show pyarrow
print(100*"-")
%pip show db-sqlite3
print(100*"-")
%pip show polars
print(100*"-")
%pip show pandas
print(100*"-")
%pip show grain
print(100*"-")
%pip show bitsandbytes
print(100*"-")
%pip show numpyro
print(100*"-")
%pip show langdetect
print(100*"-")
%pip show xprof
print(100*"-")
%pip show jax2onnx
print(100*"-")
%pip show onnx
print(100*"-")
%pip show onnxruntime
print(100*"-")
%pip show tensorboard-plugin-profile
print(100*"-")
%pip show numpy
print(100*"-")
%pip show matplotlib
print(100*"-")
%pip show scipy
print(100*"-")

----------------------------------------------------------------------------------------------------
Name: jax
Version: 0.8.2
Summary: Differentiate, compile, and transform Numpy code.
Home-page: https://github.com/jax-ml/jax
Author: JAX team
Author-email: jax-dev@google.com
License: Apache-2.0
Location: /usr/local/lib/python3.12/dist-packages
Requires: jaxlib, ml_dtypes, numpy, opt_einsum, scipy
Required-by: chex, clu, dopamine_rl, equinox, flax, gemma, jax2onnx, jraph, kauldron, numpyro, optax, orbax-checkpoint, orbax-export
Note: you may need to restart the kernel to use updated packages.
----------------------------------------------------------------------------------------------------
Name: jaxlib
Version: 0.8.2
Summary: XLA library for JAX
Home-page: https://github.com/jax-ml/jax
Author: JAX team
Author-email: jax-dev@google.com
License: Apache-2.0
Location: /usr/local/lib/python3.12/dist-packages
Requires: ml_dtypes, numpy, scipy
Required-by: chex, clu, dopamine_rl, jax, jraph,

In [3]:
print("Environment setup complete!")

Environment setup complete!


## ***Import and configuration***

In [4]:
import os
import sys
import io

import warnings
warnings.filterwarnings("ignore")

import json
import tensorflow as tf
import subprocess
import pickle
import zipfile
import base64
import shutil
import sqlite3
import numpy as np
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from tqdm import tqdm
import networkx as nx
from pathlib import Path
import functools
from functools import partial
from typing import (
    Any,
    Tuple,
    Callable,
    Optional,
    Sequence,
    List,
    Dict
)
from IPython.display import clear_output

import pyarrow as pa
import pyarrow.parquet as pq

from ogb.nodeproppred import NodePropPredDataset

# JAX and Flax NNX
import jax
import jax.ops
import jax.lax
import jax.profiler
import jax.numpy as jnp
from jax import (
    random,
    jit,
    value_and_grad
)
import jax.tree_util as tree_util
import flax.nnx as nnx
from flax.nnx import filterlib
from flax.serialization import (
    msgpack_serialize,
    from_bytes
)
import orbax.checkpoint as ocp
from orbax.checkpoint import PyTreeCheckpointer, CheckpointManager

# Optimization
import optax

import jraph

from sklearn.metrics import accuracy_score, confusion_matrix

import numpyro
import numpyro.distributions as dist
from numpyro.infer import MCMC, NUTS

from datasets import load_dataset

from kaggle_secrets import UserSecretsClient

# PyTorch for compatibility (GGUF conversion)
import torch

import jax2onnx
from jax2onnx import onnx_function, to_onnx

import onnx
import onnxruntime as ort


2025-12-22 14:52:23.382703: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766415143.830755     252 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766415143.939610     252 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766415145.070352     252 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766415145.070408     252 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766415145.070411     252 computation_placer.cc:177] computation placer alr

## ***Ensure JAX uses GPU if available***

In [5]:
# Configure JAX for GPU
try:
    jax.config.update('jax_platform_name', 'gpu')
    print("JAX devices:", jax.devices())
except RuntimeError:
    print("GPU not available, using CPU")
    jax.config.update('jax_platform_name', 'cpu')
    print("JAX devices:", jax.devices())

JAX devices: [CudaDevice(id=0), CudaDevice(id=1)]


## ***Setting up access to Hugging Face***

In [6]:
def set_git_config(email, name):
    try:
        # Setting global user.email
        subprocess.run(["git", "config", "--global", "user.email", email], check=True)
        print(f"Git user.email set to: {email}")

        # Setting the global user.name
        subprocess.run(["git", "config", "--global", "user.name", name], check=True)
        print(f"Git user.name set to: {name}")

        # Check settings (optional)
        email_output = subprocess.run(["git", "config", "--global", "user.email"], capture_output=True, text=True, check=True)
        name_output = subprocess.run(["git", "config", "--global", "user.name"], capture_output=True, text=True, check=True)
        print(f"Check - Email: {email_output.stdout.strip()}")
        print(f"Check - Name: {name_output.stdout.strip()}")

    except subprocess.CalledProcessError as e:
        print(f"Error while setting up Git configuration: {e}")

## ***Data loader (your hybrid Data Lake – SQLite + Parquet)***

In [7]:
class OGBNProteinsLoader:
    def __init__(self, db_path: str, edges_parquet: str, nodes_parquet: str):
        self.db_path = Path(db_path)
        self.edges_parquet = Path(edges_parquet)
        self.nodes_parquet = Path(nodes_parquet)
        
        # Načtení SQLite (jen pro rychlou indexaci sousedů)
        self.conn = sqlite3.connect(self.db_path)
        
        # Načtení Parquet – hlavní data
        self.edges = pq.read_table(self.edges_parquet).to_pandas()
        self.nodes = pq.read_table(self.nodes_parquet).to_pandas()
        
        # Features a labels – přímo ze sloupců 'features' a 'labels' (arrays)
        self.features = np.stack(self.nodes["features"].values).astype(np.float32)
        self.labels   = np.stack(self.nodes["labels"].values).astype(np.float32)
        
        # Node ID a indexování
        self.node_ids = self.nodes["node_id"].values
        self.node_id_to_idx = {nid: idx for idx, nid in enumerate(self.node_ids)}
        
        # === SPLIT PŘÍMO Z PARQUETU (sloupec 'split') ===
        self.train_idx = self.nodes[self.nodes["split"] == "train"]["node_id"].values
        self.val_idx   = self.nodes[self.nodes["split"] == "valid"]["node_id"].values
        self.test_idx  = self.nodes[self.nodes["split"] == "test"]["node_id"].values
        
        print(f"Dataset loaded:")
        print(f"  - Total nodes: {len(self.node_ids)}")
        print(f"  - Train: {len(self.train_idx)} | Valid: {len(self.val_idx)} | Test: {len(self.test_idx)}")
        print(f"  - Features shape: {self.features.shape}")
        print(f"  - Labels shape:   {self.labels.shape}")

    def get_neighbors(self, node_id: int, max_neighbors: int = 25):
        """Rychlá indexace z SQLite (B-tree)"""
        neighbors = set()
        neigh_source = self.edges[self.edges["source"] == node_id]["target"].values[:max_neighbors]
        neigh_target = self.edges[self.edges["target"] == node_id]["source"].values[:max_neighbors]
        neighbors.update(neigh_source)
        neighbors.update(neigh_target)
        neighbors = np.array(list(neighbors))
        return neighbors if len(neighbors) > 0 else np.array([node_id])

    def sample_batch(self, batch_size: int = 64, max_neighbors: int = 25, split: str = "train"):
        idx = {"train": self.train_idx, "valid": self.val_idx, "test": self.test_idx}[split]
        nodes = np.random.choice(idx, batch_size, replace=False)
        return [(node, self.get_neighbors(node, max_neighbors)) for node in nodes]

    def get_features(self, node_ids: np.ndarray):
        indices = [self.node_id_to_idx[nid] for nid in node_ids]
        return self.features[indices]

    def get_labels(self, node_ids: np.ndarray):
        indices = [self.node_id_to_idx[nid] for nid in node_ids]
        return self.labels[indices]

## ***Bayesian GraphSAGE v NNX + NumPyro***

In [8]:
class BayesianGraphSAGE(nnx.Module):
    def __init__(self, in_features: int = 8, hidden_features: int = 256, out_features: int = 112, rngs=None):
        self.sage1 = SAGEConv(in_features, hidden_features)
        self.sage2 = SAGEConv(hidden_features, out_features)
        self.dropout = nnx.Dropout(0.2, rngs=rngs)

    def __call__(self, x, senders, receivers, training=False):
        x = self.sage1(x, senders, receivers)
        x = jax.nn.relu(x)
        x = self.dropout(x, deterministic=not training)
        x = self.sage2(x, senders, receivers)
        return jax.nn.sigmoid(x)  # multi-label

## ***Bayesian GraphSAGE s Gradient Checkpointing***

In [9]:
class BayesianGraphSAGE(nnx.Module):
    def __init__(self, in_features=8, hidden_features=256, out_features=112, rngs=None):
        self.sage1 = SAGEConv(in_features, hidden_features)
        self.sage2 = SAGEConv(hidden_features, out_features)
        self.dropout = nnx.Dropout(0.2, rngs=rngs)

    @functools.partial(nnx.remat, static_argnums=(3,))  # Checkpointing via SAGEConv
    def sage_layer(self, x, senders, receivers, training):
        x = self.sage1(x, senders, receivers)
        x = jax.nn.relu(x)
        x = self.dropout(x, deterministic=not training)
        return x

    def __call__(self, x, senders, receivers, training=False):
        x = self.sage_layer(x, senders, receivers, training)
        x = self.sage2(x, senders, receivers)
        return jax.nn.sigmoid(x)

## ***NumPyro model + MCMC***

In [10]:
def bayesian_graphsage_model(graph_data, X, y=None):
    senders, receivers, n_nodes = graph_data
    model = BayesianGraphSAGE()
    params = model.params()
    
    new_params = {}
    for name, param in params.items():
        p_val = numpyro.sample(f"prior_{name}", dist.Normal(0.0, 1.0).to_event(2))
        new_params[name] = nnx.Param(p_val)
    
    model_sampled = model.replace(params=new_params)
    preds = model_sampled(X, senders, receivers)
    
    with numpyro.plate("data", X.shape[0]):
        numpyro.sample("obs", dist.Bernoulli(logits=jnp.log(preds / (1 - preds + 1e-10))), obs=y)

## ***Inference + Profiling + TensorBoard Logging – MCMC (NUTS)***

In [11]:
def run_bayesian_inference(loader, num_samples=500, num_chains=4):
    key = jax.random.PRNGKey(42)
    batch = loader.sample_batch(batch_size=64, max_neighbors=25)
    
    senders, receivers = [], []
    for center, neigh in batch:
        for n in neigh:
            senders.extend([center, n])
            receivers.extend([n, center])
    graph_data = (jnp.array(senders), jnp.array(receivers), len(loader.node_ids))
    
    X = loader.get_features([c for c, _ in batch])
    y = loader.get_labels([c for c, _ in batch])
    
    # TensorBoard writer for common logs
    writer = tf.summary.create_file_writer("runs/bayesian_graphsage")
    
    # Profiler – save directly to the runs/ folder
    PROFILE_DIR = "runs/bayesian_graphsage/profile"
    os.makedirs(PROFILE_DIR, exist_ok=True)  # create the folder if it doesn't exist
    
    # We run the profiler and automatically save the trace
    with jax.profiler.trace(PROFILE_DIR, create_perfetto_link=True):
        kernel = NUTS(bayesian_graphsage_model)
        mcmc = MCMC(kernel, num_warmup=500, num_samples=num_samples, num_chains=num_chains)
        start = time.time()
        mcmc.run(key, graph_data, X, y=y)
        print(f"MCMC finished in {time.time() - start:.1f} s")
        
        # Save the trace (just in case the trace wasn't saved automatically)
        jax.profiler.save_device_trace(PROFILE_DIR)
    
    # If it's in Kaggle or a temporary environment, we'll also move it to runs/
    # (mostly not needed because we already store it directly in runs/)
    # But just to be sure:
    if PROFILE_DIR.startswith("/tmp/"):
        shutil.copytree(PROFILE_DIR, "runs/bayesian_graphsage/profile", dirs_exist_ok=True)
    
    samples = mcmc.get_samples()
    
    # Log loss (as before)
    log_prob = numpyro.infer.log_likelihood(bayesian_graphsage_model, samples, graph_data, X, y=y)
    mean_log_prob = jnp.mean(log_prob)
    with writer.as_default():
        tf.summary.scalar("mcmc/mean_log_prob", float(mean_log_prob), step=0)
    writer.flush()
    
    return samples, writer

## ***Prediction with uncertainty***

In [12]:
def predict_with_uncertainty(model, samples, X, senders, receivers, num_samples=200):
    preds = []
    for i in range(num_samples):
        sample = {k: samples[k][i] for k in samples}
        model_sampled = model.replace(params=sample)
        pred = model_sampled(X, senders, receivers)
        preds.append(pred)
    preds = jnp.stack(preds, axis=0)
    return preds.mean(axis=0), preds.std(axis=0)

## ***Export modelu do ONNX a StableHLO***

In [13]:
def export_model(model: nnx.Module, sample_input: jnp.ndarray, senders: jnp.ndarray, receivers: jnp.ndarray, path_dir: str):
    """
    Exporting an NNX model to ONNX using jax2onnx
    """
    # Definuj callable pro export (NNX model je callable)
    def forward_fn(x, s, r):
        return model(x, s, r, training=False)
    
    # Prepare sample inputs (for tracing shapes)
    inputs = [sample_input, senders, receivers]
    
    # Export to ONNX (jax2onnx supports NNX directly)
    onnx_model = to_onnx(
        forward_fn,
        inputs,
        return_mode="model",  # returns onnx.ModelProto object
        input_names=["x", "senders", "receivers"],
        output_names=["predictions"]
    )
    
    # Save to file
    path_bayesian_graphsage = os.path.join(path_dir,"bayesian_graphsage.onnx")
    with open(path_bayesian_graphsage, "wb") as f:
        f.write(onnx_model.SerializeToString())
    print(f"Model exported to ONNX: {path_bayesian_graphsage} (using jax2onnx)")
    
    # Optional: for more complex models (e.g. with dropout) use decorator
    @onnx_function
    def decorated_model(x):
        return model(x, senders, receivers, training=False)
    onnx_model = to_onnx(decorated_model, [sample_input], return_mode="model")
    
    # StableHLO remains the same (if you want)
    stablehlo = serialize_stablehlo(model, sample_input, senders, receivers)
    path_stablehlo_dir = os.path.join(path_dir,"bayesian_graphsage.stablehlo")
    with open(path_stablehlo_dir, "wb") as f:
        f.write(stablehlo)
    print(f"StableHLO exported: {path_stablehlo_dir}")

## ***Main launch***

In [ ]:
if __name__ == "__main__":
    DB_PATH = os.path.join("/kaggle","input","bayesian-graphsage-dataset","datasets","graph_dataset","proteins_data.db")
    EDGES_PARQUET = os.path.join("/kaggle","input","bayesian-graphsage-dataset","datasets","graph_dataset","proteins_edges.parquet")
    NODES_PARQUET = os.path.join("/kaggle","input","bayesian-graphsage-dataset","datasets","graph_dataset","proteins_nodes.parquet")

    nodes = pq.read_table(NODES_PARQUET).to_pandas()
    print(nodes.columns.tolist())

    edges = pq.read_table(EDGES_PARQUET).to_pandas()
    print(edges.columns.tolist()) 

    path_model_dir = os.path.join('/kaggle','working','models')
    os.makedirs(path_model_dir, exist_ok=True)
    
    loader = OGBNProteinsLoader(
        db_path=DB_PATH,
        edges_parquet=EDGES_PARQUET,
        nodes_parquet=NODES_PARQUET
    )

    # Kontrola
    print(f"Prvních 5 train node_id: {loader.train_idx[:5]}")
    print(f"Prvních 5 valid node_id: {loader.val_idx[:5]}")
    
    print("Dataset loaded!")
    
    # MCMC + profiling + TensorBoard
    posterior_samples, writer = run_bayesian_inference(loader)
    
    # Validation
    val_batch = loader.sample_batch(batch_size=32, split="valid")
    senders, receivers = [], []
    for center, neigh in val_batch:
        for n in neigh:
            senders.extend([center, n])
            receivers.extend([n, center])
    senders = jnp.array(senders)
    receivers = jnp.array(receivers)
    
    val_nodes = [c for c, _ in val_batch]
    X_val = loader.get_features(val_nodes)
    y_val_true = loader.get_labels(val_nodes)
    
    model = BayesianGraphSAGE()
    mean_pred, uncertainty = predict_with_uncertainty(model, posterior_samples, X_val, senders, receivers)
    
    # Export
    export_model(model, X_val[:1], senders[:10], receivers[:10], path_dir=path_model_dir)  # small sample for export
    
    # Results
    predicted = (mean_pred > 0.5).astype(np.int32)
    true = (y_val_true > 0.5).astype(np.int32)
    acc = (predicted == true).mean() * 100
    print(f"Validation Micro-Accuracy: {acc:.2f}%")
    print(f"Average Uncertainty: {uncertainty.mean():.4f}")
    
    print("\nAll done! Check:")
    print("  - runs/bayesian_graphsage/ (TensorBoard)")
    print("  - /tmp/profile-bayesian-graphsage/ (XProf/Perfetto)")
    print("  - bayesian_graphsage.onnx (ONNX)")
    print("  - bayesian_graphsage.stablehlo (PJRT)")

['node_id', 'features', 'labels', 'split']
['source', 'target', 'feat_0', 'feat_1', 'feat_2', 'feat_3', 'feat_4', 'feat_5', 'feat_6', 'feat_7']
Dataset loaded:
  - Total nodes: 132534
  - Train: 86619 | Valid: 21236 | Test: 24679
  - Features shape: (132534, 8)
  - Labels shape:   (132534, 112)
Prvních 5 train node_id: [0 1 2 3 4]
Prvních 5 valid node_id: [86619 86620 86621 86622 86623]
Dataset loaded!


I0000 00:00:1766415192.701351     252 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2608 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1766415192.705274     252 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13940 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Open URL in browser: https://ui.perfetto.dev/#!/?url=http://127.0.0.1:9001/perfetto_trace.json.gz


## ***Model interference***

In [ ]:
model = onnx.load(os.path.join(path_model_dir,"bayesian_graphsage.onnx"))
onnx.checker.check_model(model)  # validation
print("ONNX model valid!")

In [ ]:
# Inference test
ort_session = ort.InferenceSession(os.path.join(path_model_dir,"bayesian_graphsage.onnx"))
outputs = ort_session.run(None, {"x": sample_input, "senders": senders, "receivers": receivers})
print("ONNX inference OK!")